# Movie Data Sanity Checking

by [Max Woolf](http://minimaxir.com) (@minimaxir)

*This notebook is licensed under the MIT License. If you use the code or data visualization designs contained within this notebook, it would be greatly appreciated if proper attribution is given back to this notebook and/or myself. Thanks! :)*

In [2]:
library(dplyr)
library(readr)

## Process the Data

Take the movies data, load in R friendly format, and combine with Rotten Tomatoes data.

In [7]:
df <- read_delim("~/Downloads/omdb0316/omdbMovies.txt", "\t", col_types="iccicccccccidi_c_____")
df_tomatoes <- read_delim("~/Downloads/omdb0316/tomatoes.txt",  "\t", col_types="i_diiiicidi_cc_c")
df <- df %>% left_join(df_tomatoes, by="ID")
rm(df_tomatoes)

|================================================================================| 100%  435 MB


Warning message:
: 23 problems parsing '~/Downloads/omdb0316/omdbMovies.txt'. See problems(...) for more details.

In [8]:
parseBoxOffice <- function(x) {
    unit <- 0
    if (is.na(x) | x=="") {return (NA)}
    if (substr(x, nchar(x), nchar(x)) == "k") {unit <- 10^3}
    else {unit <- 10^6}

    number <- as.numeric(substr(x,2,nchar(x)-1))

    return(number * unit)
}

df <- df %>% mutate(BoxOffice = as.numeric(sapply(BoxOffice, parseBoxOffice)), ReleaseMonth = as.numeric(substr(Released,6,7)))

Warning message:
In eval(substitute(expr), envir, enclos): NAs introduced by coercion

## Box Office

What are the best movies?

In [19]:
print(df %>% select(imdbID, Title, Year, BoxOffice) %>% arrange(desc(BoxOffice)) %>% head(25), n=25)
print(nrow(df))

Source: local data frame [25 x 4]

      imdbID                                         Title  Year BoxOffice
       (chr)                                         (chr) (int)     (dbl)
1  tt2488496    Star Wars: Episode VII - The Force Awakens  2015 930800000
2  tt0499549                                        Avatar  2009 760500000
3  tt0369610                                Jurassic World  2015 652200000
4  tt0848228                                  The Avengers  2012 623300000
5  tt0468569                               The Dark Knight  2008 533300000
6  tt1774602                               The Dark Knight  2008 533300000
7  tt2395427                       Avengers: Age of Ultron  2015 459000000
8  tt1345836                         The Dark Knight Rises  2012 448100000
9  tt0298148                                       Shrek 2  2004 436500000
10 tt1951264               The Hunger Games: Catching Fire  2013 424600000
11 tt0383574    Pirates of the Caribbean: Dead Man's Chest  2006 

In [20]:
df_dup <- df %>% select(Title, Year) %>% mutate(Title = gsub("The ", "", Title))
dup <- duplicated(df_dup)   # find entry indices which are duplicates
rm(df_dup)   # remove temp dataframe

df_dedup <- df %>% filter(!dup)   # keep entries which are *not* dups
print(df_dedup %>% select(imdbID, Title, Year, BoxOffice) %>% arrange(desc(BoxOffice)) %>% head(25), n=25)
print(nrow(df_dedup))

Source: local data frame [25 x 4]

      imdbID                                         Title  Year BoxOffice
       (chr)                                         (chr) (int)     (dbl)
1  tt2488496    Star Wars: Episode VII - The Force Awakens  2015 930800000
2  tt0499549                                        Avatar  2009 760500000
3  tt0369610                                Jurassic World  2015 652200000
4  tt0848228                                  The Avengers  2012 623300000
5  tt0468569                               The Dark Knight  2008 533300000
6  tt2395427                       Avengers: Age of Ultron  2015 459000000
7  tt1345836                         The Dark Knight Rises  2012 448100000
8  tt0298148                                       Shrek 2  2004 436500000
9  tt1951264               The Hunger Games: Catching Fire  2013 424600000
10 tt0383574    Pirates of the Caribbean: Dead Man's Chest  2006 423000000
11 tt0435761                                   Toy Story 3  2010 

In [21]:
write.csv(df_dedup %>% select(imdbID, Title, Year, BoxOffice) %>% arrange(desc(BoxOffice)) %>% head(200), "top_movies_0316_dedup.csv", row.names=F)

## Inflation

In [28]:
inflation <- read_csv("http://research.stlouisfed.org/fred2/data/CPIAUCSL.csv") %>%
                    group_by(Year = as.integer(substr(DATE, 1, 4))) %>%
                    summarize(Avg_Value = mean(VALUE)) %>%   # average across all months
                    mutate(Adjust = tail(Avg_Value,1) / Avg_Value)   # normalize by most-recent year

print(inflation %>% head())
print(inflation %>% tail())

Source: local data frame [6 x 3]

   Year Avg_Value    Adjust
  (int)     (dbl)     (dbl)
1  1947  22.33167 10.653347
2  1948  24.04500  9.894240
3  1949  23.80917  9.992244
4  1950  24.06250  9.887044
5  1951  25.97333  9.159664
6  1952  26.56667  8.955094
Source: local data frame [6 x 3]

   Year Avg_Value   Adjust
  (int)     (dbl)    (dbl)
1  2011  224.9230 1.057726
2  2012  229.5963 1.036197
3  2013  232.9636 1.021220
4  2014  236.7147 1.005037
5  2015  236.9952 1.003847
6  2016  237.9070 1.000000


In [32]:
df_dedup_join <- df_dedup %>% inner_join(inflation) %>% mutate(AdjBoxOffice = BoxOffice * Adjust)

print(df_dedup_join %>% select(Title, Year, AdjBoxOffice) %>% arrange(desc(AdjBoxOffice)) %>% head(25), n=25)

Joining by: "Year"


Source: local data frame [25 x 3]

                                               Title  Year AdjBoxOffice
                                               (chr) (int)        (dbl)
1                                          The Lorax  1972   1200678740
2         Star Wars: Episode VII - The Force Awakens  2015    934381231
3                                             Avatar  2009    843234239
4                                     Jurassic World  2015    654709324
5                                       The Avengers  2012    645861504
6                                    The Dark Knight  2008    589422987
7                                            Shrek 2  2004    549718499
8                                         Spider-Man  2002    533968065
9         Pirates of the Caribbean: Dead Man's Chest  2006    499283058
10     The Lord of the Rings: The Return of the King  2003    470642109
11                                      Spider-Man 2  2004    470251747
12                         Th

In [33]:
write.csv(df_dedup_join %>% select(imdbID, Title, Year, AdjBoxOffice) %>% arrange(desc(AdjBoxOffice)) %>% head(200), "top_movies_0316_dedup_adj.csv", row.names=F)

## The MIT License (MIT)

Copyright (c) 2016 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
